# Top 10 ən çox vergi tutuan məhsullar-> top limit istifade etmeden

In [ ]:
SELECT tax,[Stock Item]
FROM(
select max([Tax Amount]) as tax, si.[Stock Item],
DENSE_RANK() over( order by max([Tax Amount]) desc) as Rnk
from Fact.Sale as fs
inner join Dimension.[Stock Item] si
on si.[Stock Item Key] = fs.[Stock Item Key]
group by [Stock Item]
)t
where Rnk<=10 group by [Stock Item],tax order by tax desc


# Adında 3 dənə təkrar hərif olan customer-ler 

In [ ]:
with tbl1 as
(
select c.[Primary Contact],
replace(c.[Primary Contact], ' ', '')
as [Customer Name] from Dimension.Customer c
),
tbl2 as
(
select tbl1.[Customer Name], substring(tbl1.[Customer Name],1,1) as ltr,
substring(tbl1.[Customer Name],2, len(tbl1.[Customer Name])-1) as temp
from tbl1
where len(tbl1.[Customer Name])>3
union all
select tbl2.[Customer Name], substring(temp,1,1) as ltr,
substring(temp, 2, len(temp)-1) as temp
from tbl2
where len(temp)>0
)
select distinct [Customer Name]
from tbl2
group by [Customer Name], ltr
having count(ltr)=3

# 1 selectle  tarixçə boyu bütün işçilərdən gələn profit . ( Nəticədə 3 sütun - tarix , emploee_id , profit ) ( With ve temp cedvel istifadə etmədən)


In [ ]:
select Date, [WWI Employee ID],Profit from(
select fs.[Invoice Date Key] as Date,e.[WWI Employee ID], sum(fs.Profit) as Profit
from Fact.Sale as fs  
inner join Dimension.Employee as e
on fs.[Salesperson Key] = e.[Employee Key]
where [Is Salesperson] = 1
group by [WWI Employee ID],[Invoice Date Key]
)a order by Date asc, [WWI Employee ID]


# Satıcıları totalsale -ə görə 3 Kategoriyaya böl. Ən zəif satış eliyənlərin siyahısın verməlidi.

In [ ]:
create table Emps(
Employee varchar(100),
TotalSale decimal(18,2),
Rnk int
)
insert into Emps(Employee,TotalSale,Rnk)
select de.Employee, count([Bill To Customer Key]) as TotalSale,
ROW_NUMBER() over(order by count([Bill To Customer Key]) desc) as Rnk
from Fact.Sale as fs
inner join Dimension.Employee de
on de.[Employee Key] = fs.[Salesperson Key]
where [Is Salesperson] = 1
group by de.Employee

Declare @temp int
select @temp =  count(Employee) from Emps

select Employee,TotalSale from(
select Employee,TotalSale, Rnk,
case
when Rnk<(@temp*33)/100 then 'yaxsi'
when Rnk>(@temp*33)/100 and Rnk<=(@temp*66)/100 then 'orta'
when Rnk>(@temp*66)/100 then 'zeif'
end as category
from Emps
group by Employee,TotalSale,Rnk
)b where category = 'zeif'

# Ən çox satış edən 5 şəhər . Say olaraq . ( Limit , top ve sair bu tipli cutoff funkiyalardan istifadə etməden )


In [ ]:
select City,TotalSale from(
select ds.city, count([Bill To Customer Key]) as TotalSale,
DENSE_RANK() over(order by count([Bill To Customer Key]) desc) as Rnk
from Fact.Sale as fs
inner join Dimension.City ds
on ds.[City Key] = fs.[City Key]
group by ds.City
)a
where a.Rnk<=5

# Ən gec satılan top 5 məhsul.

In [ ]:
select [Stock Item],Date from(
select s.[Stock Item], fs.[Invoice Date Key] as Date,
row_number() over(order by fs.[Invoice Date Key] desc) as Rnk
from Fact.Sale as fs
inner join Dimension.[Stock Item] s
on s.[Stock Item Key] = fs.[Stock Item Key]
group by s.[Stock Item], [Invoice Date Key]
)a
where a.Rnk<=200

# Hər bir gün üzrə satıcını satış edib etmədiyinə baxılsın. hansi gun satış etməyibsə qabağına 0 yazılsın

In [ ]:
CREATE TABLE _Dates (
  d DATE,
  PRIMARY KEY (d)
)

DECLARE @dIncr DATE = '2013-01-01'
DECLARE @dEnd DATE = '2016-05-31'

WHILE ( @dIncr < @dEnd )
BEGIN
  INSERT INTO _Dates (d) VALUES( @dIncr )
  SELECT @dIncr = DATEADD(DAY, 1, @dIncr )
END

select _Dates.d,
coalesce(a.[Amy Trefl],0) as [Amy Trefl],coalesce(a.[Anthony Grosse],0) as [Anthony Grosse],
coalesce(a.[Archer Lamble],0) as [Archer Lamble],coalesce(a.[Hudson Hollinworth],0) as [Hudson Hollinworth],
coalesce(a.[Hudson Onslow],0) as [Hudson Onslow],coalesce(a.[Jack Potter],0) as[Jack Potter],
coalesce(a.[Kayla Woodcock],0) as [Kayla Woodcock],coalesce(a.[Lily Code],0)as [Kayla Woodcock],
coalesce(a.[Sophia Hinton],0) as [Sophia Hinton],coalesce(a.[Taj Shand],0) as [Taj Shand]from _Dates
left join(
SELECT * FROM
(
select fs.[Invoice Date Key] as Date,e.Employee, sum(fs.Profit) as Profit
from Fact.Sale as fs  
inner join Dimension.Employee as e
on fs.[Salesperson Key] = e.[Employee Key]
where [Is Salesperson] = 1
group by Employee,[Invoice Date Key]
)a PIVOT(sum(Profit) FOR Employee IN([Amy Trefl],[Lily Code],[Kayla Woodcock],[Sophia Hinton],[Archer Lamble],
[Anthony Grosse],[Hudson Onslow],[Taj Shand],[Hudson Hollinworth],[Jack Potter])) AS PivotTable
)a
on _Dates.d = a.Date
order by _Dates.d


# Ən çox satış  edən 3-cü yerə layiq satıcılar .

In [ ]:
select Employee,TotalSale from(
select de.Employee, count([Bill To Customer Key]) as TotalSale,
DENSE_RANK() over(order by count([Bill To Customer Key]) desc) as Rnk
from Fact.Sale as fs
inner join Dimension.Employee de
on de.[Employee Key] = fs.[Salesperson Key]
group by de.Employee
)a
where a.Rnk=3

# Tarixcə boyu satılma sayı ən az olan 5 məhsul siayhısı

In [ ]:
select [Stock Item],TotalSale
from(
select si.[Stock Item], count([Invoice Date Key]) as TotalSale,
Dense_rank() over(order by count([Invoice Date Key]) asc) as Rnk
from Fact.Sale as fs
inner join Dimension.[Stock Item] si
on si.[Stock Item Key] = fs.[Stock Item Key]
group by si.[Stock Item]
)a
where Rnk<=5

# Profiti ən az olan 5 İşci 

In [ ]:
select Employee, Profit
from(
select e.Employee, sum(sq.Profit) as Profit,
Row_Number() over(order by sum(Profit) asc) as Rnk
from Dimension.Employee as e
inner join Fact.Sale as sq on e.[Employee Key] = sq.[Salesperson Key]
where [Is Salesperson] = 1
group by Employee
)t where Rnk<=5